<a href="https://colab.research.google.com/github/hwon-n/section4_project/blob/main/curse_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 욕설 감지 프로젝트

데이터셋 주소: [2runo github](https://github.com/2runo/Curse-detection-data)

In [6]:
pip install beautifulsoup4

In [1]:
pip install requests

In [13]:
import os
root = '/content/drive/MyDrive/curse_dataset.txt'

txt_file = open(root, encoding = 'utf-8')

curse = []
for txt in txt_file:
  txt = txt.replace('\n', '')
  text, label = txt.split('|', 1)
  curse.append([text, label])

len(curse)

5856

In [ ]:
curse

In [50]:
# LoL 인벤 자유게시판 제목 스크래핑
import requests
from bs4 import BeautifulSoup

titles = []

url = 'https://www.inven.co.kr/board/lol/4626?p='
for i in range(1, 21):
  response = requests.get(url+str(i))
  if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find_all('a', class_='subject-link')
    for i in title:
      i = i.get_text()
      i = i[5:]
      i = i.strip()
      if len(i) > 2:
        titles.append(i)

In [51]:
len(titles)

1339

In [52]:
# 파일 저장

path = '/content/drive/MyDrive/inven_data.txt'
with open(path, 'w', encoding = 'utf-8') as f:
  for title in titles:
    f.write(title+ '|1\n')

print('저장 완료')

저장 완료
